In [1]:
import os, sys, torch, time
import numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer
np.set_printoptions(precision=4, suppress=True, linewidth=200)
import torch
print(torch.__version__)
print(torch.version.cuda)

# set these before import RWKV
os.environ['RWKV_JIT_ON'] = '1'
os.environ["RWKV_CUDA_ON"] = '1' # '1' to compile CUDA kernel (10x faster), requires c++ compiler & cuda libraries
CHAT_LANG = 'Chinese'
from infer.rwkv.model import RWKV # pip install rwkv
from infer.rwkv.utils import PIPELINE, PIPELINE_ARGS

model = RWKV(model='/home/rwkv/JL/out_model/pretrain/rwkv-18', strategy='cuda fp16')
#model ='rwkv'
pipeline = PIPELINE(model, "rwkv_vocab_v20230424")


args = PIPELINE_ARGS(temperature = 1.0, top_p = 0.0, top_k=0, # top_k = 0 then ignore
                     alpha_frequency = 0.0,
                     alpha_presence = 0.0,
                     token_ban = [0], # ban the generation of some tokens
                     token_stop = [24], # stop generation whenever you see any token here
                     chunk_len = 256) # split input into chunks to save VRAM (shorter -> slower)

########################################################################################################
from world.speech_encoder import SpeechEncoder

speech_encoder = SpeechEncoder(
    '/home/rwkv/JL/audio',
    768,
    downsample_K=5,
    hidden_dim=2048,
    train_mode="adapter",
    device='cuda',
)


/home/rwkv/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2.4.1+cu121
12.1


Using /home/rwkv/.cache/torch_extensions/py310_cu121 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/rwkv/.cache/torch_extensions/py310_cu121/wkv_cuda/build.ninja...
/home/rwkv/miniconda3/lib/python3.10/site-packages/torch/utils/cpp_extension.py:1965: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
Building extension module wkv_cuda...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


ninja: no work to do.

### RWKV-7 "Goose" enabled ###



Loading extension module wkv_cuda...
Using /home/rwkv/.cache/torch_extensions/py310_cu121 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/rwkv/.cache/torch_extensions/py310_cu121/wkv7s/build.ninja...
/home/rwkv/miniconda3/lib/python3.10/site-packages/torch/utils/cpp_extension.py:1965: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
Building extension module wkv7s...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
Loading extension module wkv7s...
/home/rwkv/JL/world/infer/rwkv/model.py:244: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpi

ninja: no work to do.
Loading /home/rwkv/JL/out_model/pretrain/rwkv-18 (cuda fp16)



KeyboardInterrupt: 

In [ ]:
import io
import soundfile as sf
# # 读取parquet文件
# df = pd.read_parquet('/home/rwkv/JL/audio/test.parquet')
def bytes_to_audio(audio_bytes):
    with io.BytesIO(audio_bytes) as buf:
        # 使用 soundfile 读取音频数据
        audio_array, sr = sf.read(buf)
        
        # 确保是单声道
        if len(audio_array.shape) > 1:
            audio_array = audio_array.mean(axis=1)
        
        # 确保是 float32 类型
        audio_array = audio_array.astype(np.float32)
        
        return {
            'array': audio_array,
            'sampling_rate': sr
        }

msg = 'Assistant:'

# 方法1：使用 pandas
import pandas as pd
import librosa
import io
import soundfile as sf
# # 读取parquet文件

df = pd.read_parquet('/home/rwkv/JL/audio/test.parquet')
for idx in range(7):
    print(df['answer'][idx])
    audio_data = bytes_to_audio(df['question_audio'][idx]['bytes'])
    audio = librosa.resample(audio_data['array'],orig_sr= 48000,target_sr= 16000)
    y, _ = speech_encoder(audio)
    result = pipeline.generate(msg, token_count=500, args=args, callback=None, state=None, sign=y)
    print(result)

In [2]:
import pyaudio
import wave

def record_audio(filename="output.wav", seconds=5):
    # 设置参数
    CHUNK = 1024                # 每个缓冲区的帧数
    FORMAT = pyaudio.paInt16    # 采样格式
    CHANNELS = 1                # 单声道
    RATE = 44100               # 采样率 (Hz)
    
    # 创建 PyAudio 对象
    p = pyaudio.PyAudio()
    
    print(f"开始录音 {seconds} 秒...")
    
    # 打开音频流
    stream = p.open(format=FORMAT,
                    channels=CHANNELS,
                    rate=RATE,
                    input=True,
                    frames_per_buffer=CHUNK)
    
    # 录制音频
    frames = []
    for i in range(0, int(RATE / CHUNK * seconds)):
        data = stream.read(CHUNK)
        frames.append(data)
    
    print("录音结束!")
    
    # 停止并关闭音频流
    stream.stop_stream()
    stream.close()
    p.terminate()
    
    # 保存为 WAV 文件
    wf = wave.open(filename, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    wf.close()
    
    print(f"音频已保存为: {filename}")

# 使用示例：录制5秒钟
record_audio(seconds=5)


start recording...


ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1334:(snd_func_refer) error evaluating name
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5701:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM sysdefault
ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_concat returned error: No